### Unbound knapsack
Allowed to pick multiple copies of the same element.
Rest is same as 0/1 knapsack

##### Brute force
```for each item 'i' 
  create a new set which includes one quantity of item 'i' if it does not exceed the capacity, and 
     recursively call to process all items 
  create a new set without item 'i', and recursively process the remaining items 
return the set from the above two sets with higher profit ```

In [1]:
def solve_knapsack(profits, weights, capacity):
    return solve_knapsack_recursive(profits, weights, capacity, 0)


def solve_knapsack_recursive(profits, weights, capacity, currentIndex):
    n = len(profits)
    # base checks
    if capacity <= 0 or n == 0 or len(weights) != n or currentIndex >= n:
        return 0

    # recursive call after choosing the items at the currentIndex, note that we recursive 
    # call on all items as we did not increment currentIndex
    profit1 = 0
    if weights[currentIndex] <= capacity:
        profit1 = profits[currentIndex] + solve_knapsack_recursive(profits, weights, capacity - weights[currentIndex], currentIndex)

    # recursive call after excluding the element at the currentIndex
    profit2 = solve_knapsack_recursive(profits, weights, capacity, currentIndex + 1)

    return max(profit1, profit2)

In [2]:
print(solve_knapsack([15, 50, 60, 90], [1, 3, 4, 5], 8))
print(solve_knapsack([15, 50, 60, 90], [1, 3, 4, 5], 6))

140
105


##### Memoization
The indices in this case would be index and the capacity, therefore:

In [3]:
def solve_knapsack(profits, weights, capacity):
    dp = [[-1 for _ in range(capacity+1)] for _ in range(len(profits))]
    
    return solve_knapsack_recursive(dp, profits, weights, capacity, 0)


def solve_knapsack_recursive(dp, profits, weights, capacity, currentIndex):
    n = len(profits)
    
    # base checks
    if capacity <= 0 or n == 0 or len(weights) != n or currentIndex >= n:
        return 0

    # check if we have not already processed a similar sub-problem
    if dp[currentIndex][capacity] == -1:
        # recursive call after choosing the items at the currentIndex, note that we
        # recursive call on all items as we did not increment currentIndex
        profit1 = 0
        if weights[currentIndex] <= capacity:
            profit1 = profits[currentIndex] + solve_knapsack_recursive(dp, profits, weights, capacity - weights[currentIndex], currentIndex)

        # recursive call after excluding the element at the currentIndex
        profit2 = solve_knapsack_recursive(dp, profits, weights, capacity, currentIndex + 1)

        dp[currentIndex][capacity] = max(profit1, profit2)

    return dp[currentIndex][capacity]

In [4]:
print(solve_knapsack([15, 50, 60, 90], [1, 3, 4, 5], 8))
print(solve_knapsack([15, 50, 60, 90], [1, 3, 4, 5], 6))

140
105


##### Bottom up:
So for every possible capacity ‘c’ (0 <= c <= capacity), we have two options:

1. Exclude the item. In this case, we will take whatever profit we get from the sub-array excluding this item: dp[index-1][c]
2. Include the item if its weight is not more than the ‘c’. In this case, we include its profit plus whatever profit we get from the remaining capacity: profit[index] + dp[index][c-weight[index]]

``` dp[index][c] = max (dp[index-1][c], profit[index] + dp[index][c-weight[index]])```

In [6]:
def solve_knapsack(profits, weights, capacity):
    n = len(profits)
    # base checks
    if capacity <= 0 or n == 0 or len(weights) != n:
        return 0

    dp = [[-1 for _ in range(capacity+1)] for _ in range(len(profits))]

    # populate the capacity=0 columns
    for i in range(n):
        dp[i][0] = 0

    # process all sub-arrays for all capacities
    for i in range(n):
            for c in range(1, capacity+1):
                profit1, profit2 = 0, 0
                if weights[i] <= c:
                    profit1 = profits[i] + dp[i][c - weights[i]]
                if i > 0:
                    profit2 = dp[i - 1][c]
                dp[i][c] = profit1 if profit1 > profit2 else profit2

    # maximum profit will be in the bottom-right corner.
    return dp[n - 1][capacity]

In [7]:
print(solve_knapsack([15, 50, 60, 90], [1, 3, 4, 5], 8))
print(solve_knapsack([15, 50, 60, 90], [1, 3, 4, 5], 6))

140
105


<br>

#### Rod cutting
Given a rod length and list of profits per rod length, cut it to maximize profit

##### Brute force:
On analysis, this is exactly similar to the previous problem

```for each rod length 'i' 
  create a new set which includes one quantity of length 'i', and recursively process 
      all rod lengths for the remaining length 
  create a new set without rod length 'i', and recursively process remaining rod lengths
return the set from the above two sets with a higher sales price```

##### Bottom up:

So for every possible rod length ‘len’ (0<= len <= n), we have two options:

1. Exclude the piece. In this case, we will take whatever price we get from the rod length excluding this piece => dp[index-1][len]
2. Include the piece if its length is not more than ‘len’. In this case, we include its price plus whatever price we get from the remaining rod length => prices[index] + dp[index][len-lengths[index]]

```dp[index][len] = max (dp[index-1][len], prices[index] + dp[index][len-lengths[index]]) ```

In [8]:
def solve_rod_cutting(lengths, prices, n):
    lengthCount = len(lengths)
    # base checks
    if n <= 0 or lengthCount == 0 or len(prices) != lengthCount:
        return 0

    dp = [[0 for _ in range(n+1)] for _ in range(lengthCount)]

    # process all rod lengths for all prices
    for i in range(lengthCount):
        for length in range(1, n+1):
            p1, p2 = 0, 0
            if lengths[i] <= length:
                p1 = prices[i] + dp[i][length - lengths[i]]
            if i > 0:
                p2 = dp[i - 1][length]
            dp[i][length] = max(p1, p2)

    # maximum price will be at the bottom-right corner.
    return dp[lengthCount - 1][n]

In [9]:
print(solve_rod_cutting([1, 2, 3, 4, 5], [2, 6, 7, 10, 13], 5))

14


#### Coin change
Given set of available coins and an amount, find number of ways to find the change

_This is not an optimization problem but a counting problem_

##### Brute force:
```for each coin 'c' 
  create a new set which includes one quantity of coin 'c' if it does not exceed 'T', and 
     recursively call to process all coins 
  create a new set without coin 'c', and recursively call to process the remaining coins 
return the count of sets who have a sum equal to 'T'```

In [10]:
def count_change(denominations, total):
    return count_change_recursive(denominations, total, 0)


def count_change_recursive(denominations, total, currentIndex):
    # base checks
    if total == 0:
        return 1

    n = len(denominations)
    if n == 0 or currentIndex >= n:
        return 0

    # recursive call after selecting the coin at the currentIndex
    # if the coin at currentIndex exceeds the total, we shouldn't process this
    sum1 = 0
    if denominations[currentIndex] <= total:
        sum1 = count_change_recursive(denominations, total - denominations[currentIndex], currentIndex)

    # recursive call after excluding the coin at the currentIndex
    sum2 = count_change_recursive(denominations, total, currentIndex + 1)

    return sum1 + sum2

In [11]:
print(count_change([1, 2, 3], 5))

5


##### Memoization:
The indices here would be total and currentIndex

In [12]:
def count_change(denominations, total):
    dp = [[-1 for _ in range(total+1)] for _ in range(len(denominations))]
    return count_change_recursive(dp, denominations, total, 0)


def count_change_recursive(dp, denominations, total, currentIndex):
    # base checks
    if total == 0:
        return 1

    n = len(denominations)
    if n == 0 or currentIndex >= n:
        return 0

    if dp[currentIndex][total] != -1:
        return dp[currentIndex][total]

    # recursive call after selecting the coin at the currentIndex
    # if the coin at currentIndex exceeds the total, we shouldn't process this
    sum1 = 0
    if denominations[currentIndex] <= total:
        sum1 = count_change_recursive(dp, denominations, total - denominations[currentIndex], currentIndex)

    # recursive call after excluding the coin at the currentIndex
    sum2 = count_change_recursive(dp, denominations, total, currentIndex + 1)

    dp[currentIndex][total] = sum1 + sum2
    
    return dp[currentIndex][total]

In [13]:
print(count_change([1, 2, 3], 5))

5


##### Bottom up:
for every possible total ‘t’ (0<= t <= Total) and for every possible coin index (0 <= index < denominations.length), we have two options:

1. Exclude the coin. Count all the coin combinations without the given coin up to the total ‘t’ => dp[index-1][t]
2. Include the coin if its value is not more than ‘t’. In this case, we will count all the coin combinations to get the remaining total: dp[index][t-denominations[index]]

```dp[index][t] = dp[index-1][t] + dp[index][t-denominations[index]]```

In [14]:
def count_change(denominations, total):
    n = len(denominations)
    dp = [[0 for _ in range(total+1)] for _ in range(n)]

    # populate the total = 0 columns, as we will always have an empty set for zero total
    for i in range(n):
        dp[i][0] = 1

    # process all sub-arrays for all capacities
    for i in range(n):
        for t in range(1, total+1):
            if i > 0:
                dp[i][t] = dp[i - 1][t]
            if t >= denominations[i]:
                dp[i][t] += dp[i][t - denominations[i]]

    # total combinations will be at the bottom-right corner.
    return dp[n - 1][total]

In [15]:
print(count_change([1, 2, 3], 5))

5


#### Minimum coin change:
An extension of the previous problem. Here we need to minimise the number of coins required

##### Brute force:
```for each coin 'c' 
  create a new set which includes one quantity of coin 'c' if it does not exceed 'T', and 
     recursively call to process all coins 
  create a new set without coin 'c', and recursively call to process the remaining coins 
return the count of coins from the above two sets with a smaller number of coins```

In [16]:
import math


def count_change(denominations, total):
    result = count_change_recursive(denominations, total, 0)
    return -1 if result == math.inf else result


def count_change_recursive(denominations, total, currentIndex):
    # base check
    if total == 0:
        return 0

    n = len(denominations)
    if n == 0 or currentIndex >= n:
        return math.inf

    # recursive call after selecting the coin at the currentIndex
    # if the coin at currentIndex exceeds the total, we shouldn't process this
    count1 = math.inf
    if denominations[currentIndex] <= total:
        res = count_change_recursive(denominations, total - denominations[currentIndex], currentIndex)
        if res != math.inf:
            count1 = res + 1

    # recursive call after excluding the coin at the currentIndex
    count2 = count_change_recursive(denominations, total, currentIndex + 1)

    return min(count1, count2)

In [17]:
print(count_change([1, 2, 3], 5))
print(count_change([1, 2, 3], 11))
print(count_change([1, 2, 3], 7))
print(count_change([3, 5], 7))

2
4
3
-1


##### Memoization:
The indices here will be current item and total (same as prev)
Only difference will be what is being returned

In [18]:
import math


def count_change(denominations, total):
    dp = [[-1 for _ in range(total+1)] for _ in range(len(denominations))]
    result = count_change_recursive(dp, denominations, total, 0)
    
    return -1 if result == math.inf else result


def count_change_recursive(dp, denominations, total, currentIndex):
    # base check
    if total == 0:
        return 0
    n = len(denominations)
    if n == 0 or currentIndex >= n:
        return math.inf

    # check if we have not already processed a similar sub-problem
    if dp[currentIndex][total] == -1:
        # recursive call after selecting the coin at the currentIndex
        # if the coin at currentIndex exceeds the total, we shouldn't process this
        count1 = math.inf
        if denominations[currentIndex] <= total:
            res = count_change_recursive(dp, denominations, total - denominations[currentIndex], currentIndex)
            if res != math.inf:
                count1 = res + 1

        # recursive call after excluding the coin at the currentIndex
        count2 = count_change_recursive(dp, denominations, total, currentIndex + 1)
        
        dp[currentIndex][total] = min(count1, count2)

    return dp[currentIndex][total]

In [19]:
print(count_change([1, 2, 3], 5))
print(count_change([1, 2, 3], 11))
print(count_change([1, 2, 3], 7))
print(count_change([3, 5], 7))

2
4
3
-1


##### Bottom up:
for every possible total ‘t’ (0<= t <= Total) and for every possible coin index (0 <= index < denominations.length), we have two options:
    
1. Exclude the coin: In this case, we will take the minimum coin count from the previous set => dp[index-1][t]
2. Include the coin if its value is not more than ‘t’: In this case, we will take the minimum count needed to get the remaining total, plus include ‘1’ for the current coin => dp[index][t-denominations[index]] + 1

```dp[index][t] = min(dp[index-1][t], dp[index][t-denominations[index]] + 1)```

In [20]:
import math


def count_change(denominations, total):
    n = len(denominations)
    dp = [[math.inf for _ in range(total+1)] for _ in range(n)]

    # populate the total=0 columns, as we don't need any coin to make zero total
    for i in range(n):
        dp[i][0] = 0

    for i in range(n):
        for t in range(1, total+1):
            if i > 0:
                dp[i][t] = dp[i - 1][t]  # exclude the coin
            if t >= denominations[i]:
                if dp[i][t - denominations[i]] != math.inf:
                    # include the coin
                    dp[i][t] = min(dp[i][t], dp[i][t - denominations[i]] + 1)

    # total combinations will be at the bottom-right corner.
    return -1 if dp[n - 1][total] == math.inf else dp[n - 1][total]

In [21]:
print(count_change([1, 2, 3], 5))
print(count_change([1, 2, 3], 11))
print(count_change([1, 2, 3], 7))
print(count_change([3, 5], 7))

2
4
3
-1


#### Maximum ribbon cut:
Given long ribbon and possibe cut lengths, obtain maximum number of pieces

##### Brute force:
```for each length 'l' 
  create a new set which includes one quantity of length 'l' if it does not exceed 'n',
     and recursively call to process all lengths 
  create a new set without length 'l', and recursively process the remaining lengths
return the number of pieces from the above two sets with a higher number of pieces```

In [22]:
import math


def count_ribbon_pieces(ribbonLengths, total):
    maxPieces = count_ribbon_pieces_recursive(ribbonLengths, total, 0)
    return -1 if maxPieces == -math.inf else maxPieces


def count_ribbon_pieces_recursive(ribbonLengths, total, currentIndex):
    # base check
    if total == 0:
        return 0

    n = len(ribbonLengths)
    if n == 0 or currentIndex >= n:
        return -math.inf

    # recursive call after selecting the ribbon length at the currentIndex
    # if the ribbon length at the currentIndex exceeds the total, we shouldn't process this
    c1 = -math.inf
    if ribbonLengths[currentIndex] <= total:
        result = count_ribbon_pieces_recursive(ribbonLengths, total - ribbonLengths[currentIndex], currentIndex)
        if result != -math.inf:
            c1 = result + 1

    # recursive call after excluding the ribbon length at the currentIndex
    c2 = count_ribbon_pieces_recursive(ribbonLengths, total, currentIndex + 1)
    return max(c1, c2)

In [23]:
print(count_ribbon_pieces([2, 3, 5], 5))
print(count_ribbon_pieces([2, 3], 7))
print(count_ribbon_pieces([3, 5, 7], 13))
print(count_ribbon_pieces([3, 5], 7))

2
3
3
-1


##### Bottom up:
Very similar to minimum coin change, except here it is the max

for every possible length ‘len’ (0 <= len <= total) and for every possible ribbon length index (0 <= index < ribbonLengths.length), we have two options:
    
1. Exclude the ribbon length: In this case, we will take the maximum piece count from the previous set => dp[index-1][len]
2. Include the ribbon length if its value is not more than ‘len’: In this case, we will take the maximum pieces needed to get the remaining total, plus include ‘1’ for the current ribbon length => 1 + dp[index][len-ribbonLengths[index]]

```dp[index][len] = max(dp[index-1][len], 1 + dp[index][len-ribbonLengths[index]])```

In [24]:
import math


def count_ribbon_pieces(ribbonLengths, total):
    n = len(ribbonLengths)
    dp = [[-math.inf for _ in range(total+1)] for _ in range(n)]

    # populate the total=0 columns, as we don't need any ribbon to make zero total
    for i in range(n):
        dp[i][0] = 0

    for i in range(n):
        for t in range(1, total+1):
            if i > 0:  # exclude the ribbon
                dp[i][t] = dp[i - 1][t]
            # include the ribbon and check if the remaining length can be cut into 
            # available lengths
            if t >= ribbonLengths[i] and dp[i][t - ribbonLengths[i]] != -math.inf:
                dp[i][t] = max(dp[i][t], dp[i][t - ribbonLengths[i]] + 1)

    # total combinations will be at the bottom-right corner, return '-1' if cutting is 
    # not possible
    return -1 if dp[n - 1][total] == -math.inf else dp[n - 1][total]

In [25]:
print(count_ribbon_pieces([2, 3, 5], 5))
print(count_ribbon_pieces([2, 3], 7))
print(count_ribbon_pieces([3, 5, 7], 13))
print(count_ribbon_pieces([3, 5], 7))

2
3
3
-1
